In [1]:
import openpyxl
from collections import defaultdict

In [2]:
orders_file = 'data/orders.xlsx'
workbook = openpyxl.load_workbook(orders_file)

In [3]:
sheetnames= workbook.get_sheet_names()
print(sheetnames)

['Microsofts', 'Netflix']


C:\Users\Thumbby\AppData\Local\Temp\ipykernel_3060\2214667223.py:1: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  sheetnames= workbook.get_sheet_names()


In [4]:
sheetMSFT = workbook.worksheets[0]
sheetNFLX = workbook.worksheets[1]
sheetMSFTNAME = sheetMSFT.title
sheetNFLXNAME = sheetNFLX.title
print(sheetMSFTNAME)
print(sheetNFLXNAME)

Microsofts
Netflix


In [5]:
excelbids = {}
excelbids['BATS'] = {}
excelbids['BATS'][sheetMSFTNAME] = defaultdict(list)
excelbids['BATS'][sheetNFLXNAME] = defaultdict(list)
excelbids['NASD'] = {}
excelbids['NASD'][sheetMSFTNAME] = defaultdict(list)
excelbids['NASD'][sheetNFLXNAME] = defaultdict(list)
excelasks = {}
excelasks['BATS'] = {}
excelasks['BATS'][sheetMSFTNAME] = defaultdict(list)
excelasks['BATS'][sheetNFLXNAME] = defaultdict(list)
excelasks['NASD'] = {}
excelasks['NASD'][sheetMSFTNAME] = defaultdict(list)
excelasks['NASD'][sheetNFLXNAME] = defaultdict(list)


In [6]:
# Get the dictionary of price-to-qunatity
for i in range (3, 7):
	for j in range (1, 2):
		excelbids['NASD'][sheetMSFTNAME][sheetMSFT.cell(i, j).value].append(sheetMSFT.cell(i, j+1).value)
for i in range (3, 7):
	for j in range (3, 4):
		excelasks['NASD'][sheetMSFTNAME][sheetMSFT.cell(i, j).value].append(sheetMSFT.cell(i, j+1).value)

for i in range (10, 14):
		for j in range (1, 2):
			excelbids['BATS'][sheetMSFTNAME][sheetMSFT.cell(i, j).value].append(sheetMSFT.cell(i, j+1).value)
for i in range (10, 14):
	for j in range (3, 4):
		excelasks['BATS'][sheetMSFTNAME][sheetMSFT.cell(i, j).value].append(sheetMSFT.cell(i, j+1).value)

for i in range (3, 7):
	for j in range (1, 2):
		excelbids['NASD'][sheetNFLXNAME][sheetNFLX.cell(i, j).value].append(sheetNFLX.cell(i, j+1).value)

for i in range (3, 7):
	for j in range (3, 4):
		excelasks['NASD'][sheetNFLXNAME][sheetNFLX.cell(i, j).value].append(sheetNFLX.cell(i, j+1).value)

for i in range (10, 14):
	for j in range (1, 2):
		excelbids['BATS'][sheetNFLXNAME][sheetNFLX.cell(i, j).value].append(sheetNFLX.cell(i, j+1).value)

for i in range (10, 14):
	for j in range (3, 4):
		excelasks['BATS'][sheetNFLXNAME][sheetNFLX.cell(i, j).value].append(sheetNFLX.cell(i, j+1).value)

print(excelbids)
print(excelasks)

{'BATS': {'Microsofts': defaultdict(<class 'list'>, {183.65: [30], 183.64: [450], 183.63: [175], 183.62: [400]}), 'Netflix': defaultdict(<class 'list'>, {366.95: [110, 200], 366.94: [250], 366.93: [375]})}, 'NASD': {'Microsofts': defaultdict(<class 'list'>, {183.65: [500], 183.64: [200], 183.63: [500], 183.62: [1450]}), 'Netflix': defaultdict(<class 'list'>, {366.97: [100], 366.96: [200], 366.95: [225], 366.94: [450]})}}
{'BATS': {'Microsofts': defaultdict(<class 'list'>, {183.91: [500], 183.94: [400], 183.95: [1125], 183.96: [675]}), 'Netflix': defaultdict(<class 'list'>, {367.23: [250], 367.24: [300], 367.25: [100], 367.35: [100]})}, 'NASD': {'Microsofts': defaultdict(<class 'list'>, {183.91: [100], 183.92: [300], 183.93: [1075], 183.94: [800]}), 'Netflix': defaultdict(<class 'list'>, {367.23: [600], 367.24: [575], 367.25: [425], 367.26: [325]})}}


In [7]:
def smart_order_router(side, symbol, qty, order_type='market', price=None):
    needed_qty = qty
    route = {}
    available_qty = 0
    if side == 'buy':
        for exchanges in excelasks:
            if qty <= 0:
                break
            available_orders = excelasks[exchanges]
            if symbol in available_orders.keys():
                target_exchange = exchanges
                for ask_price in available_orders[symbol].keys():
                    if order_type == 'market' or price >= ask_price:
                        for ask_qty in available_orders[symbol][ask_price]:
                            if target_exchange not in route:
                                route[target_exchange] = {}
                            if ask_qty >= qty and qty > 0:
                                route[target_exchange][ask_price] = qty
                                qty = 0
                            elif qty > 0:
                                route[target_exchange][ask_price] = ask_qty
                                qty = qty - ask_qty
                            available_qty += ask_qty
            else:
                pass
    elif side == 'sell':
        for exchanges in excelbids:
            if qty <= 0:
                break
            available_orders = excelbids[exchanges]
            if symbol in available_orders.keys():
                target_exchange = exchanges
                for bid_price in available_orders[symbol].keys():
                    if order_type == 'market' or price >= bid_price:
                        for bid_qty in available_orders[symbol][bid_price]:
                            if target_exchange not in route:
                                route[target_exchange] = {}
                            if bid_qty >= qty and qty > 0:
                                route[target_exchange][bid_price] = qty
                                qty = 0
                            elif qty > 0:
                                route[target_exchange][bid_price] = bid_qty
                                qty = qty - bid_qty
                            available_qty += ask_qty
            else:
                pass
    order = {'route':route, 'leave_qty':needed_qty - available_qty}
    return order

In [8]:
smart_order_router('buy', 'Microsofts', 1000, 'limit', 183.92)

{'route': {'BATS': {183.91: 500}, 'NASD': {183.91: 100, 183.92: 300}},
 'leave_qty': 100}